In [36]:
import os
import math
import operator
import json

from collections import Counter

import numpy as np
import pandas as pd

from scipy.io import wavfile
from pydub import AudioSegment
from audio.processor import WavProcessor, format_predictions

from log_config import LOGGING

def get_labels(csv_file = 'labels.csv'): 
    df = pd.read_csv(csv_file)
    df = df.set_index('index')
    return df

def get_label_val(idx): 
    return [0.0 if val != 1 else 1.0 for val in df.loc[idx].values]

def label_sig(sig, df = get_labels()):
    window_size = 6 * 1000
    overlap = 3 * 1000
    fragment_length = 900 * 1000

    current_length = 0

    window_start = 0
    window_end = window_start + window_size

    #Audio labels predicted by the algorithm for each window length.
    sig_labels = []

    while current_length < fragment_length:
        print(window_start,window_end)

        segment = sig[window_start:window_end]
        samples = np.array(segment.get_array_of_samples())
        new_sig = samples.astype(np.float32)

        with WavProcessor() as proc:
            predictions = proc.get_predictions(16000, new_sig)

            for p in predictions:
                sig_labels.append(p)
                print(str(window_start / (60*1000)), ":", str(window_end / (60*1000)), " - ", df.loc[p[0]].values[1], p[1])

        current_length = window_start
        window_start += (window_size - overlap)
        window_end = (window_start + window_size)
        
    return sig_labels

def classify_labels(labels, df = get_labels()):
    class_dict = {}

    total_items = 0

    for l in labels:
        total_items += 1

        if l[0] not in class_dict:
            class_dict[l[0]] = []

        class_dict[l[0]].append(l[1])

    class_dict_harmonic = {}

    for key, val in class_dict.items():
        class_prob = len(val)/float(total_items)
        class_prob = 1 / (class_prob)
        confidence_prob = 1 / np.mean(val)
        harmonic = 2 / ( class_prob + confidence_prob )
        class_dict_harmonic[key] = harmonic

    sorted_harmonics = sorted(class_dict_harmonic.items(), key=lambda x: x[1])

    classification_harmonics = {}

    df_labels = list(df)[2:]

    df_without_nan = df.dropna(thresh=3)
    total_labels = df_without_nan.shape[0]

    for label in df_labels:
        prob = []
        for key, val in sorted_harmonics:
            if not (math.isnan(df.loc[key][label])):
                prob.append(val)
            else:
                prob.append(0.0)
        harmonic = 2 / (1 / (df_without_nan[label].count()/total_labels) + 1 / np.mean(prob))
        classification_harmonics[label] = harmonic

    classification_harmonics_sorted = sorted(classification_harmonics.items(), key=lambda x: x[1])

    for key, val in classification_harmonics_sorted:
        print(key,val)
        
    return classification_harmonics_sorted

def load_wav_from_folder(wav_folder = "wav"):
    signals = []

    for file in os.listdir(wav_folder):
        if file.endswith(".wav"):
            path = os.path.join(wav_folder, file)
            sig = load_wav(path)
            labels = label_sig(sig)
            res = classify_labels(labels)
            
            with open('result/' + str(file) + '_6_3_900.json', 'w') as f:
                json.dump(res, f)
                
            print('Saved ' + str(file) + '_6_3_900.json')
            
    return signals

def load_wav(path):
    sig = AudioSegment.from_file(path, format="wav")
    return sig

load_wav_from_folder()

0 6000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.0 : 0.1  -  Vehicle 0.18370972573757172
0.0 : 0.1  -  Music 0.10150744020938873
3000 9000


Exception ignored in: <generator object get_controller at 0x127788468>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 4971, in get_controller
    yield g
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 4789, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.05 : 0.15  -  Vehicle 0.2727358639240265
6000 12000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.1 : 0.2  -  Vehicle 0.8922986388206482
0.1 : 0.2  -  Medium engine (mid frequency) 0.14632706344127655
0.1 : 0.2  -  Static 0.1019400879740715
9000 15000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
12000 18000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.2 : 0.3  -  Vehicle 0.24179509282112122
15000 21000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
18000 24000
INFO:tensorflow:Restoring parameters 

1.9 : 2.0  -  Car passing by 0.3807666301727295
117000 123000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
1.95 : 2.05  -  Car passing by 0.1647437959909439
120000 126000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
2.0 : 2.1  -  Static 0.1405070424079895
2.0 : 2.1  -  Car passing by 0.13028883934020996
123000 129000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
2.05 : 2.15  -  Car passing by 0.1871030330657959
126000 132000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
129000 135000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
132000 138000
INFO

4.05 : 4.15  -  Vehicle 0.17076869308948517
246000 252000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
4.1 : 4.2  -  Vehicle 0.23222990334033966
4.1 : 4.2  -  Frog 0.1768997460603714
249000 255000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
252000 258000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
4.2 : 4.3  -  Vehicle 0.2617551386356354
4.2 : 4.3  -  Frog 0.14612649381160736
255000 261000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
258000 264000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
4.3 : 4.4  -  Vehicle 0.1301533281803131
261000 267

...

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
8.4 : 8.5  -  Frog 0.18735136091709137
507000 513000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
510000 516000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
8.5 : 8.6  -  Frog 0.14554312825202942
513000 519000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
8.55 : 8.65  -  Music 0.4765580892562866
516000 522000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
519000 525000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
5

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
633000 639000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
10.55 : 10.65  -  Frog 0.13741041719913483
636000 642000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
10.6 : 10.7  -  Frog 0.10846807062625885
639000 645000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
642000 648000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
10.7 : 10.8  -  Chime 0.11072652041912079
645000 651000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
10.75 : 10.85  -  Music 0.40254998207092285
10.75 : 10.85  - 

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
12.45 : 12.55  -  Frog 0.8907257318496704
12.45 : 12.55  -  Bird 0.7127330899238586
12.45 : 12.55  -  Croak 0.45712196826934814
12.45 : 12.55  -  Bird vocalization, bird call, bird song 0.40606889128685
12.45 : 12.55  -  Chirp, tweet 0.23450785875320435
750000 756000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
12.5 : 12.6  -  Frog 0.5253772735595703
12.5 : 12.6  -  Croak 0.10654523968696594
753000 759000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
12.55 : 12.65  -  Frog 0.43810486793518066
12.55 : 12.65  -  Croak 0.19250977039337158
756000 762000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
12.6 : 12.7  -  Wind chime 0.39864805340766907
12.6 : 12.7  - 

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
14.45 : 14.55  -  Fowl 0.7870537638664246
870000 876000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
14.5 : 14.6  -  Fowl 0.3882555663585663
14.5 : 14.6  -  Whistling 0.1334967464208603
873000 879000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
14.55 : 14.65  -  Whistling 0.1299535036087036
876000 882000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
879000 885000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
14.65 : 14.75  -  Fowl 0.1447668969631195
882000 888000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters fr

[]